# 案例说明：多闸门河流 PID 控制

本案例模拟了一个带有反馈控制的河流系统。该系统由一个上游水库、一个中间河道以及两个分别位于水库下游和河道下游的闸门组成。两个闸门均由独立的PID控制器进行自动控制。

这个案例的主要目的是：
- 演示如何在一个仿真案例中配置和使用控制器 (`Controller`)。
- 展示如何通过`config.json`文件定义控制器参数及其“接线”（即控制器观测哪个组件的哪个状态，并控制另一个组件）。
- 验证`SimulationHarness`仿真器对闭环控制系统的支持能力。

### 1. 导入库并加载配置

In [ ]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt

from swp.simulation_identification.physical_objects.reservoir import Reservoir
from swp.simulation_identification.physical_objects.gate import Gate
from swp.simulation_identification.physical_objects.river_channel import RiverChannel
from swp.local_agents.control.pid_controller import PIDController
from swp.core_engine.testing.simulation_harness import SimulationHarness

CONFIG_PATH = 'examples/multi_gate_river.json'
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

print("配置加载成功！")

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

### 2. 定义并运行仿真

In [ ]:
def run_multi_gate_river(cfg):
    """根据配置搭建并运行包含控制器的多闸门河流仿真。"""
    print(f"--- 开始从配置文件加载仿真: {CONFIG_PATH} ---")

    COMPONENT_CLASS_MAP = {"Reservoir": Reservoir, "Gate": Gate, "RiverChannel": RiverChannel}
    CONTROLLER_CLASS_MAP = {"PIDController": PIDController}

    # 1. 设置仿真平台
    harness = SimulationHarness(config=cfg['simulation_settings'])

    # 2. 实例化并添加物理组件
    print("\n--- 正在实例化物理组件 ---")
    for comp_conf in cfg['components']:
        CompClass = COMPONENT_CLASS_MAP[comp_conf['type']]
        instance = CompClass(name=comp_conf['id'], initial_state=comp_conf.get('initial_state', {}), parameters=comp_conf.get('params', {}))
        harness.add_component(instance)
    
    # 3. 添加连接
    print("\n--- 正在连接组件 ---")
    for conn in cfg['connections']: harness.add_connection(conn['from'], conn['to'])

    # 4. 实例化并添加控制器
    print("\n--- 正在实例化控制器 ---")
    for ctrl_conf in cfg.get('controllers', []):
        CtrlClass = CONTROLLER_CLASS_MAP[ctrl_conf['type']]
        controller = CtrlClass(**ctrl_conf['params'])
        wiring = ctrl_conf['wiring']
        harness.add_controller(ctrl_conf['id'], controller, wiring['controlled_id'], wiring['observed_id'], wiring['observation_key'])
        print(f"  - 已创建控制器: {ctrl_conf['id']}")
        
    # 5. 构建并运行仿真
    harness.build()
    harness.run_simulation()

    # 6. 保存结果
    output_path = os.path.join(os.path.dirname(CONFIG_PATH), "multi_gate_river_output.json")
    with open(output_path, 'w') as f: json.dump(harness.history, f, indent=4)
    print(f"\n--- 结果已保存至 {output_path} ---")
    return harness.history

history = run_multi_gate_river(config)

### 3. 可视化结果

In [ ]:
def plot_controller_results(simulation_history):
    if not simulation_history:
        print("没有历史数据可供绘制。")
        return

    df = pd.DataFrame()
    for step in simulation_history:
        time = step['time']
        row = {'time': time}
        for comp, states in step.items():
            if comp == 'time': continue
            for state, value in states.items(): row[f"{comp}.{state}"] = value
        df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    df = df.set_index('time')

    fig, axes = plt.subplots(3, 1, figsize=(15, 15), sharex=True)
    fig.suptitle('多闸门河流PID控制结果', fontsize=16)

    # 图1: 上游水库水位与闸门1开度
    ax1_twin = axes[0].twinx()
    axes[0].plot(df.index, df['reservoir_1.water_level'], 'b-', label='水库水位 (m)')
    axes[0].axhline(y=18.0, color='r', linestyle=':', label='目标水位 (18.0m)')
    ax1_twin.plot(df.index, df['gate_1.opening'], 'g--', label='闸门1开度')
    axes[0].set_title('控制器1: 维持上游水库水位')
    axes[0].set_ylabel('水位 (m)', color='b'); ax1_twin.set_ylabel('开度', color='g')
    axes[0].legend(loc='upper left'); ax1_twin.legend(loc='upper right')

    # 图2: 中间河道蓄水量与闸门2开度
    ax2_twin = axes[1].twinx()
    axes[1].plot(df.index, df['channel_1.volume'], 'b-', label='河道蓄水量 (m³)')
    axes[1].axhline(y=400000, color='r', linestyle=':', label='目标蓄水量 (400,000m³)')
    ax2_twin.plot(df.index, df['gate_2.opening'], 'g--', label='闸门2开度')
    axes[1].set_title('控制器2: 维持河道蓄水量')
    axes[1].set_ylabel('蓄水量 (m³)', color='b'); ax2_twin.set_ylabel('开度', color='g')
    axes[1].legend(loc='upper left'); ax2_twin.legend(loc='upper right')
    
    # 图3: 流量
    axes[2].plot(df.index, df['gate_1.outflow'], label='闸门1出流')
    axes[2].plot(df.index, df['gate_2.outflow'], label='闸门2出流')
    axes[2].set_title('系统流量')
    axes[2].set_ylabel('流量 (m³/s)')
    axes[2].set_xlabel('时间 (s)')
    axes[2].legend()

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

plot_controller_results(history)